## TextBlob

In [126]:
# https://textblob.readthedocs.io/en/dev/

# How to install TextBlob
#     1. pip install -U textblob
#     2. python -m textblob.download_corpora

from textblob import TextBlob

blob = TextBlob(dfT['text'][0]) # run json-to-pandas

# Part-of-speech Tagging
print blob.tags
print

# Noun Phrase Extraction¶
print blob.noun_phrases
print 

# Tokenization
print blob.words

NameError: name 'dfT' is not defined

In [39]:
# The subjectivity is a float within the range [0.0, 1.0] 
# where 0.0 is very objective and 1.0 is very subjective
for sentence in blob.sentences:
    print sentence.sentiment.polarity
print

# Lemmatize each word
for sentence in blob.sentences:    
    for word in sentence.words:
        print "%s---%s" % (word, word.lemmatize('v')) # 'v' for 'verb'

0.0166666666667

One---One
of---of
China---China
's---'s
first---first
female---female
fighter---fighter
pilots---pilot
was---be
killed---kill
in---in
a---a
training---train
accident---accident
according---accord
to---to
state-run---state-run
media---media
reports…---reports…
https---https
t.co/DoEZLme8Cq---t.co/DoEZLme8Cq


## NLTK

http://www.nltk.org/

In [10]:
import nltk
from nltk.corpus import treebank # to draw a parse tree

sentence = dfT['text'][0] # run json-to-pandas

tokens = nltk.word_tokenize(sentence)

tagged = nltk.pos_tag(tokens)

# Identify named entities - Make parse tree?
# You might need to call nltk.download() and down load some packages
entities = nltk.chunk.ne_chunk(tagged)

NameError: name 'dfT' is not defined

In [12]:
# Example from http://streamhacker.com/2010/05/10/text-classification-sentiment-analysis-naive-bayes-classifier/
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
 
def word_feats(words):
    return dict([(word, True) for word in words])
 
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')
 
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]
 
negcutoff = len(negfeats)*3/4
poscutoff = len(posfeats)*3/4
 
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print 'train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats))
 
classifier = NaiveBayesClassifier.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)
classifier.show_most_informative_features()

train on 1500 instances, test on 500 instances
accuracy: 0.728
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


In [46]:
# Example from http://streamhacker.com/2010/05/10/text-classification-sentiment-analysis-naive-bayes-classifier/
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
 
def word_feats(words):
    return dict([(word, True) for word in words])
 
negfeats = [(word_feats(text.split()), 'neg') for text in sample.loc[sample['labels'] == -1]['texts']]
#neufeats = [(word_feats(text.split()), 'neu') for text in sample.loc[sample['labels'] == 0]['texts']]
posfeats = [(word_feats(text.split()), 'pos') for text in sample.loc[sample['labels'] == 1]['texts']]

negcutoff = int(math.ceil(len(negfeats)*0.92)) # 92%
#neucutoff = int(math.ceil(len(neufeats)*0.92)) # 92%
poscutoff = int(math.ceil(len(posfeats)*0.92)) # 92%
 
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]# + neufeats[:neucutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]# + neufeats[neucutoff:]
print 'train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats))
 
classifier = NaiveBayesClassifier.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)
classifier.show_most_informative_features()

train on 665 instances, test on 56 instances
accuracy: 0.767857142857
Most Informative Features
                Carolina = True              pos : neg    =     10.3 : 1.0
                 project = True              pos : neg    =      9.1 : 1.0
                    love = True              pos : neg    =      8.7 : 1.0
                  attack = True              neg : pos    =      8.5 : 1.0
                 history = True              pos : neg    =      7.2 : 1.0
               community = True              pos : neg    =      7.2 : 1.0
                   Kaine = True              pos : neg    =      7.2 : 1.0
                     NBC = True              pos : neg    =      7.2 : 1.0
         President-elect = True              pos : neg    =      7.2 : 1.0
                     Tim = True              pos : neg    =      7.2 : 1.0


In [30]:
most_informative = {"lead":16.7/1, 
                    "primary":8.7/1, 
                    "projects":8.7/1, 
                    "voters":8.1/1, 
                    "Ohio":7.2/1, 
                    "NBC":7.2/1, 
                    "President-elect":7.2/1, 
                    "CBS":7.2/1, 
                    "love":7.2/1, 
                    "News":7.2/1}
print sample['texts'].str.count("primary").sum()
print sample['texts'].str.count("lead").sum()

43
66


## Supervised Learning w/ manual scripting

In [36]:
# Task 1: Load the texts
import pandas as pd
import glob, os         # for reading all .txt files
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
#from sklearn.base import ClassifierMixin
#from sklearn.dummy import DummyClassifier
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OutputCodeClassifier
#from sklearn.neural_network import MLPClassifier
#from sklearn.calibration import CalibratedClassifierCV
#from sklearn.semi_supervised import LabelPropagation
import math
import scipy.sparse as sp
from nltk.corpus import stopwords # Import the stop word list
from textblob import TextBlob
from datetime import datetime
import time

In [37]:
# THREE ANSWERS TO ONE
sample = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/Turk/6source_results_filtered.csv') 
#sample['Answer.sentiment'] = sample['Answer.sentiment'].map({'Positive':1, 'Neutral':0, 'Negative':-1})

new_index = range(0, len(sample.index)/3)
new_sample = pd.DataFrame(index=new_index, columns=['texts', 'sources', 'labels', 'statusID'])

text = ""
source = ""
statusID = ""
#acc_value = 0 # accumulated score per post
#num_posts = 0 # number of same text posts
pos = 0 # number of 'positive' per post
neu = 0 # number of 'neutral' per post
neg = 0 # number of 'negative' per post
pd_index = 0

for index, row in sample.iterrows():
    # If this new post is different post, then append what we worked so far
    if text != row['Input.content']:
        if text != "":
            if pos >= 2:
                new_sample.loc[[pd_index], ['labels']] = 1
            elif neu >= 2: # could be commented out for pos/neg
                new_sample.loc[[pd_index], ['labels']] = 0
            elif neg >= 2:
                new_sample.loc[[pd_index], ['labels']] = -1
            else: # 1:1:1
                #continue # discard
                pd_index -= 1 # to keep the same index

            #new_sample.loc[[pd_index], ['score']] = acc_value*1.0/num_posts
            new_sample.loc[[pd_index], ['sources']] = source
            new_sample.loc[[pd_index], ['texts']] = text
            new_sample.loc[[pd_index], ['statusID']] = statusID
            pd_index += 1
            
        # Assign a new post
        text = row['Input.content']
        source = row['Input.source']
        statusID = row['Input.statis_id']
        #acc_value = 0
        #num_posts = 0
        pos = 0
        neu = 0
        neg = 0
        
    if row['Answer.sentiment'] == "Positive":
        #acc_value += 1
        #num_posts += 1
        pos += 1
    elif row['Answer.sentiment'] == "Neutral":
        #num_posts += 1
        neu += 1
    elif row['Answer.sentiment'] == "Negative":
        #acc_value -= 1
        #num_posts += 1
        neg += 1

# not perfect, so we have some extra NaN rows (all should be filled, technically)
new_sample = new_sample.dropna()
        
# change 0.333, 0.666, ... to whole number by x 3
#new_sample.loc[:, 'score'] *= 3
new_sample['labels'] = new_sample['labels'].astype('int')
    
#new_sample['sources'] = new_sample['sources'].map({'msnbc':4, 
#                                                   'cnn':3, 
#                                                   'cbs':2,
#                                                   'usatoday':1,
#                                                   'wsj':-1,
#                                                   'fox':-2})
#new_sample['sources'] = new_sample['sources'].astype('int')
print sample.shape
sample = new_sample

#cutoff = int(math.ceil(len(sample.index)*0.9)) # 90%
#train = pd.DataFrame({'labels':sample['labels'][:cutoff], 'texts':sample['texts'][:cutoff], 'sources':sample['sources'][:cutoff]})
#test  = pd.DataFrame({'labels':sample['labels'][cutoff:], 'texts':sample['texts'][cutoff:], 'sources':sample['sources'][cutoff:]})
#print train.shape
#print test.shape
print sample.shape
print sample

(4500, 5)
(1402, 4)
                                                  texts   sources  labels  \
0     It is unusual for a president's children to be...       cnn       1   
1     Just hours after Mitt Romney blasted Donald J....       cbs      -1   
2     Hillary Clinton wins the state of Washington, ...       cbs       1   
3     "We have to take the jobs away from other coun...       fox      -1   
4     Jessica Leeds, 74, of Manhattan, was one of tw...  usatoday      -1   
5     Ash Carter warns Russia its policy in Syria wi...       cbs      -1   
6     "It's Rubio!" Watch Donald J. Trump use a wate...       fox       0   
7     Online, some Trump supporters discuss monitori...  usatoday      -1   
8     This mogul once said he's a member of "the luc...       cnn       1   
9     Live: Hillary Clinton speaks at the NAACP Conv...       fox       0   
10    BREAKING: Hillary Clinton has chosen Virginia ...       cbs       0   
11    The billboard translated reads: "Donald Trump,... 

In [38]:
# classifying candidates
def classify_post (row):
   status = row['texts'].lower()
   if (('donald' in status) or ('trump' in status)) and (('hillary' in status) or ('clinton' in status)) :
      return 'other'
   elif ('donald' in status) or ('trump' in status) :
      return 'trump'
   elif ('hillary' in status) or ('clinton' in status) :
      return 'clinton'
   return 'other'

sample['candidates'] = sample.apply (lambda row: classify_post (row),axis=1)

print sample.head()

                                               texts   sources  labels  \
0  It is unusual for a president's children to be...       cnn       1   
1  Just hours after Mitt Romney blasted Donald J....       cbs      -1   
2  Hillary Clinton wins the state of Washington, ...       cbs       1   
3  "We have to take the jobs away from other coun...       fox      -1   
4  Jessica Leeds, 74, of Manhattan, was one of tw...  usatoday      -1   

                         statusID candidates  
0    5550296508_10155606385896509      trump  
1  131459315949_10153427911075950      trump  
2  131459315949_10154056641215950    clinton  
3   15704546335_10153385284896336      trump  
4   13652355666_10153933280450667      trump  


In [39]:
raw = {}
raw['cbs'] = pd.read_csv('/Users/Haru/Documents/! college/4. Fall 2016/489Project/Facebook_RAW/CBSNews_facebook_statuses.csv')
raw['cnn'] = pd.read_csv('/Users/Haru/Documents/! college/4. Fall 2016/489Project/Facebook_RAW/cnn_facebook_statuses.csv', encoding='utf-8')
raw['fox'] = pd.read_csv('/Users/Haru/Documents/! college/4. Fall 2016/489Project/Facebook_RAW/FoxNews_facebook_statuses.csv', encoding='utf-8')
raw['msnbc'] = pd.read_csv('/Users/Haru/Documents/! college/4. Fall 2016/489Project/Facebook_RAW/msnbc_facebook_statuses.csv', encoding='utf-8') 
raw['nyt'] = pd.read_csv('/Users/Haru/Documents/! college/4. Fall 2016/489Project/Facebook_RAW/nytimes_facebook_statuses.csv', encoding='utf-8') 
raw['usatoday'] = pd.read_csv('/Users/Haru/Documents/! college/4. Fall 2016/489Project/Facebook_RAW/usatoday_facebook_statuses.csv', encoding='utf-8') 
raw['wsj'] = pd.read_csv('/Users/Haru/Documents/! college/4. Fall 2016/489Project/Facebook_RAW/wsj_facebook_statuses.csv', encoding='utf-8') 

sample['dates'] = ""

for index, row in sample.iterrows():
    the_source = row['sources']
    
    raw_index = list(raw[the_source]['status_id']).index(row['statusID'])
    sample['dates'][index] = raw[the_source]['status_published'][raw_index]

print sample.dropna().shape
print sample.shape
print sample.head()

/Users/Haru/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(1402, 6)
(1402, 6)
                                               texts   sources  labels  \
0  It is unusual for a president's children to be...       cnn       1   
1  Just hours after Mitt Romney blasted Donald J....       cbs      -1   
2  Hillary Clinton wins the state of Washington, ...       cbs       1   
3  "We have to take the jobs away from other coun...       fox      -1   
4  Jessica Leeds, 74, of Manhattan, was one of tw...  usatoday      -1   

                         statusID candidates                dates  
0    5550296508_10155606385896509      trump  2016-11-15 00:00:02  
1  131459315949_10153427911075950      trump  2016-03-03 14:44:04  
2  131459315949_10154056641215950    clinton  2016-11-09 00:03:03  
3   15704546335_10153385284896336      trump  2015-06-23 02:00:01  
4   13652355666_10153933280450667      trump  2016-10-13 21:42:41  


In [40]:
sample['date'] = ""
sample['hour'] = ""

for index, row in sample.iterrows():
    date_obj = datetime.strptime(row['dates'], "%Y-%m-%d %H:%M:%S")
    sample['date'][index] = date_obj.strftime("%Y-%m")
    sample['hour'][index] = date_obj.strftime("%H")

sample.head()

/Users/Haru/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Haru/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,texts,sources,labels,statusID,candidates,dates,date,hour
0,It is unusual for a president's children to be...,cnn,1,5550296508_10155606385896509,trump,2016-11-15 00:00:02,2016-11,00
1,Just hours after Mitt Romney blasted Donald J....,cbs,-1,131459315949_10153427911075950,trump,2016-03-03 14:44:04,2016-03,14
2,"Hillary Clinton wins the state of Washington, ...",cbs,1,131459315949_10154056641215950,clinton,2016-11-09 00:03:03,2016-11,00
3,"""We have to take the jobs away from other coun...",fox,-1,15704546335_10153385284896336,trump,2015-06-23 02:00:01,2015-06,02
4,"Jessica Leeds, 74, of Manhattan, was one of tw...",usatoday,-1,13652355666_10153933280450667,trump,2016-10-13 21:42:41,2016-10,21


In [41]:
# Lemmanize the sentences (e.g. is, are, am ->> be)
# Might be better to skip it

new_index = range(0, len(sample.index))
new_sample = pd.DataFrame(index=new_index, columns=("texts", "sources", "labels"))
i=0
for text in sample['texts']:
    blob = TextBlob(text)
    newtexts = ""

    for sentence in blob.sentences:
        newtext = ""

        for word in sentence.words:
            newtext += " " + word.lemmatize('v') # 'v' for 'verb'

        newtexts += newtext
    new_sample['texts'].loc[i] = newtexts
    i += 1
    
sample['texts'] = new_sample['texts']
print sample

                                                  texts   sources  labels  \
0      It be unusual for a president 's children to ...       cnn       1   
1      Just hours after Mitt Romney blast Donald J T...       cbs      -1   
2      Hillary Clinton win the state of Washington C...       cbs       1   
3      We have to take the job away from other count...       fox      -1   
4      Jessica Leeds 74 of Manhattan be one of two w...  usatoday      -1   
5      Ash Carter warn Russia its policy in Syria wi...       cbs      -1   
6      It 's Rubio Watch Donald J Trump use a water ...       fox       0   
7      Online some Trump supporters discuss monitor ...  usatoday      -1   
8      This mogul once say he 's a member of the luc...       cnn       1   
9      Live Hillary Clinton speak at the NAACP Conve...       fox       0   
10     BREAKING Hillary Clinton have choose Virginia...       cbs       0   
11     The billboard translate read Donald Trump he ...  usatoday      -1   

In [42]:
# Getting rid of too common words (e.g. the, an)

#print stopwords.words("english") 

new_index = range(0, len(sample.index))
new_sample = pd.DataFrame(index=new_index, columns=("texts", "sources", "labels"))

i=0
for index, row in sample.iterrows():
    # If this new post is different post, then append what we worked so far
    words = [w for w in row['texts'].split() if not w in stopwords.words("english")]
    new_sample['texts'].loc[i] = ' '.join(words)
    i += 1

sample['texts'] = new_sample['texts']
print sample

                                                  texts   sources  labels  \
0     It unusual president 's children clue White Ho...       cnn       1   
1     Just hours Mitt Romney blast Donald J Trump sp...       cbs      -1   
2     Hillary Clinton win state Washington CBS News ...       cbs       1   
3     We take job away countries make product 're ta...       fox      -1   
4     Jessica Leeds 74 Manhattan one two women quote...  usatoday      -1   
5     Ash Carter warn Russia policy Syria consequenc...       cbs      -1   
6     It 's Rubio Watch Donald J Trump use water bot...       fox       0   
7     Online Trump supporters discuss monitor poll p...  usatoday      -1   
8     This mogul say 's member lucky sperm club How ...       cnn       1   
9     Live Hillary Clinton speak NAACP Convention Ci...       fox       0   
10    BREAKING Hillary Clinton choose Virginia Senat...       cbs       0   
11    The billboard translate read Donald Trump ca n...  usatoday      -1   

In [30]:
#for j in range(0,2): # stop_words: k=0 'engl' k=1 none

cutoff = int(math.ceil(len(sample.index)*0.9)) # 90%
train = pd.DataFrame({'labels':sample['labels'][:cutoff], 
                      'texts':sample['texts'][:cutoff], 
                      'sources':sample['sources'][:cutoff], 
                      'candidates':sample['candidates'][:cutoff], 
                      'date':sample['date'][:cutoff], 
                      'hour':sample['hour'][:cutoff]})
test  = pd.DataFrame({'labels':sample['labels'][cutoff:], 
                      'texts':sample['texts'][cutoff:], 
                      'sources':sample['sources'][cutoff:], 
                      'candidates':sample['candidates'][cutoff:], 
                      'date':sample['date'][cutoff:], 
                      'hour':sample['hour'][cutoff:]})

best_score = 0

for h in range(5,18,2): # min_df from 0.05 to 0.30, adding 0.02 every time
    for i in range(95,72,-2): # max_df from 0.95 to 0.70, -0.02 every time
        for k in range(0,2): # k=0 CountVectorizer (count), k=1 TfidfVectorizer (weighed)
            for l in range(0,25): 
                for m in range(300,13001,1000): # max_features (how many most frequent words should we consider)

                    if k==0:
                        tf_vectorizer = CountVectorizer(max_df=i/100.0, 
                                                        min_df=h/100.0, 
                                                        max_features = m)
                    elif k==1:
                        tf_vectorizer = TfidfVectorizer(max_df=i/100.0, 
                                                        min_df=h/100.0, 
                                                        max_features = m)

                    train_text_tf_  = tf_vectorizer.fit_transform(train['texts'].values)
                    test_text_tf_  = tf_vectorizer.transform(test['texts'].values)

                    train_source_tf_ = tf_vectorizer.fit_transform(train['sources'].values)
                    test_source_tf_ = tf_vectorizer.transform(test['sources'].values)

                    train_candidate = tf_vectorizer.fit_transform(train['candidates'].values)
                    test_candidate = tf_vectorizer.transform(test['candidates'].values)

                    train_date = tf_vectorizer.fit_transform(train['date'].values)
                    test_date = tf_vectorizer.transform(test['date'].values)
                    
                    train_hour = tf_vectorizer.fit_transform(train['hour'].values)
                    test_hour = tf_vectorizer.transform(test['hour'].values)
                    ####
                    train_combined = sp.hstack([train_text_tf_, train_source_tf_], format='csr')
                    test_combined = sp.hstack([test_text_tf_, test_source_tf_], format='csr')

                    train_combined = sp.hstack([train_combined, train_candidate], format='csr')
                    test_combined = sp.hstack([test_combined, test_candidate], format='csr')

                    train_combined = sp.hstack([train_combined, train_date], format='csr')
                    test_combined = sp.hstack([test_combined, test_date], format='csr')

                    train_combined = sp.hstack([train_combined, train_hour], format='csr')
                    test_combined = sp.hstack([test_combined, test_hour], format='csr')

                    # CHANGE ARGS TOO
                    if l==0:
                        clf = MultinomialNB()
                    elif l==1:
                        clf = GaussianNB()
                    elif l==2:
                        clf = BernoulliNB()
                    elif l==3:
                        clf = LogisticRegression(random_state=1)
                    elif l==4:
                        clf = LogisticRegressionCV()
                    elif l==5:
                        clf = SGDClassifier()
                    elif l==6:
                        clf = Ridge()
                    elif l==7:
                        clf = RidgeClassifier()
                    elif l==8:
                        clf = RidgeClassifierCV()
                    elif l==9:
                        clf = ElasticNet()
                    elif l==10:
                        clf = LassoLars()
                    elif l==11:
                        clf = PassiveAggressiveClassifier()
                    elif l==12:
                        clf = SVC(random_state=1)
                    elif l==13:
                        clf = LinearSVC()
                    elif l==14:
                        clf = KNeighborsClassifier(n_neighbors=9)
                    elif l==15:
                        clf = NearestCentroid()
                    elif l==16:
                        clf = GradientBoostingClassifier()
                    elif l==17:
                        clf = BaggingClassifier(random_state=1)
                    elif l==18:
                        clf = RandomForestClassifier(n_estimators=100, random_state=1)
                    elif l==19:
                        clf = AdaBoostClassifier()
                    elif l==20:
                        clf = ExtraTreesClassifier()
                    elif l==21:
                        clf = GradientBoostingClassifier()
                    elif l==22:
                        clf = VotingClassifier(estimators=
                                [('l01', MultinomialNB()),
                                ('l02', GaussianNB()),
                                ('l03', BernoulliNB()),
                                ('l04', LogisticRegression(random_state=1)),
                                ('l13', SVC(random_state=1)),
                                ('l15', KNeighborsClassifier(n_neighbors=9)),
                                ('l18', BaggingClassifier(random_state=1)),
                                ('l19', RandomForestClassifier(n_estimators=100, random_state=1)),
                                ('l24', DecisionTreeClassifier(random_state=1))])
                    elif l==23:
                        clf = ExtraTreeClassifier()
                    elif l==24:
                        clf = DecisionTreeClassifier(random_state=1)

                    clf.fit(train_combined.toarray(), train['labels'])
                    score = clf.score(test_combined.toarray(), test['labels'])
                    if score > best_score:
                        best_score = score
                        print "%d %d %d %d %d\t %.4f" % (h,i,k,l,m,score)
        print "%d %d %d %d %d" % (h,i,k,l,m)

5 95 0 0 300	 0.2013
5 95 0 4 300	 0.2416
5 95 0 5 12300	 0.2685
5 95 1 24 12300
5 93 0 11 9300	 0.2752


KeyboardInterrupt: 

Out of the 20,000+ possibilities and running 8000+ cases, <br>
It never hit 0.7.. ->> For train:test = 1:1 size

Train:test = 10:1 size. I do get up to 0.8+ <br>
Now we also have up to (more reasonable) 14400 cases (all expected to run)

## Supervised Learning w/ TPOT

In [37]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import csv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
#for i in range(0,2): # 'Negative' from sample = -1 or 0
#  if i==0:
#      sample['Answer.sentiment'] = sample['Answer.sentiment'].map({'Positive':1, 'Negative':-1})
#  elif i==1:
#      sample['Answer.sentiment'] = sample['Answer.sentiment'].map({'Positive':1, 'Negative':0})

for j in range(5,21,2): # min_df
    for k in range(80,101,2): # max_df
        for l in range(0,2): # l=0 CountVectorizer (count), l=1 TfidfVectorizer (weighed)
            for m in range(50,101,10): # train:test = m/100 : (1-m/100) so 50:50 to 90:10
                for n in range(5,7): # generation (# of TPOT iteration)
                    for p in range(20,26,5): # pop_size p 
                        for q in range(10,12): # k-fold number
                            print ""    

                            #sample = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/sample_data_results.csv') 

                            # label ('positive','Negative') ->> (#,#) (e.g. (1,-1) or (1,0))
                            #sample['Answer.sentiment'] = sample['Answer.sentiment'].map({'Positive':1, 'Neutral':0, 'Negative':-1})

                            if l==0:
                                tf_vectorizer = CountVectorizer(min_df=j/100.0, max_df=k/100.0, max_features=5000)
                            elif l==1:
                                tf_vectorizer = TfidfVectorizer(min_df=j/100.0, max_df=k/100.0, max_features=5000)

                            #sample_input_tf  = tf_vectorizer.fit_transform(sample['Input.content'].values)
                            text_tf_  = tf_vectorizer.fit_transform(sample['texts'].values)
                            source_tf_ = tf_vectorizer.fit_transform(sample['sources'].values)
                            candidates_tf_ = tf_vectorizer.fit_transform(sample['candidates'].values)
                            date_tf_ = tf_vectorizer.fit_transform(sample['date'].values)
                            hour_tf_ = tf_vectorizer.fit_transform(sample['hour'].values)

                            combined = sp.hstack([text_tf_, source_tf_], format='csr')
                            combined = sp.hstack([combined, candidates_tf_], format='csr')
                            combined = sp.hstack([combined, date_tf_], format='csr')
                            combined = sp.hstack([combined, hour_tf_], format='csr')

                            X_train, X_test, y_train, y_test = train_test_split(combined, sample['labels'].values,
                                                                train_size=m/100.0, test_size=(1-m/100.0))#, random_state=)

                            # Official website example: gen=5, pop_size=20, verbo=2
                            tpot = TPOTClassifier(generations=n, population_size=p, num_cv_folds=q,
                                                  verbosity=2)
                            tpot.fit(X_train, y_train)
                            print(tpot.score(X_test, y_test))
                            print "%d %d %d %d %d %d %d %d" % (i,j,k,l,m,n,p,q)

Optimization Progress:  18%|█▊        | 21/120 [00:15<01:01,  1.60pipeline/s]

Generation 1 - Current best internal CV score: 0.567128522453


Optimization Progress:  35%|███▌      | 42/120 [00:21<00:11,  6.74pipeline/s]

Generation 2 - Current best internal CV score: 0.567128522453


Optimization Progress:  52%|█████▏    | 62/120 [01:07<03:11,  3.30s/pipeline]

...

Optimization Progress:  84%|████████▍ | 101/120 [01:50<00:08,  2.28pipeline/s]

Generation 5 - Current best internal CV score: 0.567894748329



Best pipeline: DecisionTreeClassifier(input_matrix)
0.505894778536
1402 5 80 0 50 5 20 11




GP closed prematurely - will use current best pipeline


ValueError: A pipeline has not yet been optimized. Please call fit() first.

CountVectorizer, Pos=1 Neg=-1, train:test=75:25, size=872 in TPOT = 0.78 <br>
CountVectorizer, Pos=1 Neg=-1, train:test=92:08, size=872 in TPOT = 0.78 <br>
CountVectorizer, Pos=1 Neg =0, train:test=92:08, size=872 in TPOT = 0.86 <br>



## Unsupervised Learning

In [35]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


from sklearn.decomposition import NMF, LatentDirichletAllocation
import pandas as pd
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

df1 = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/Facebook_RAW/CBSNews_facebook_statuses.csv') 
df2 = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/Facebook_RAW/cnn_facebook_statuses.csv') 
df3 = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/Facebook_RAW/facebook_sample_data.txt') 
df4 = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/Facebook_RAW/FoxNews_facebook_statuses.csv') 
df5 = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/Facebook_RAW/msnbc_facebook_statuses.csv') 
df6 = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/Facebook_RAW/nytimes_facebook_statuses.csv') 
df7 = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/Facebook_RAW/usatoday_facebook_statuses.csv') 
df8 = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/Facebook_RAW/wsj_facebook_statuses.csv') 

df1.append(df2)
df3.append(df4)
df5.append(df6)
df7.append(df8)

df1.append(df3)
df5.append(df7)

df1.append(df5)

sample = df1

#print sample.head()

cutoff = int(math.ceil(len(sample.index)*0.92)) # 92%
train = pd.DataFrame({'texts':sample['status_message'][:cutoff]})
test  = pd.DataFrame({'texts':sample['status_message'][cutoff:]})

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, analyzer = "word",
                                   #max_features=n_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(train['texts'].values)

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, analyzer = "word",
                                #max_features=n_features,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(train['texts'].values)

lda = LatentDirichletAllocation(#n_topics=n_topics, 
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)

tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)

Topic #0:
matter lives series black super city moon mckinnon photo parks landing photoshopped tweeted kate kill coalition spray concluded blasts camera
Topic #1:
carson ben suggesting damage rise dr faced backlash caused later khan health trump care hero views polls insults young rhetoric
Topic #2:
clinton hillary trump donald today eye opener matters said campaign world 90 seconds benghazi face emails presidential debate gave lawmakers
Topic #3:
trump donald cbsn clinton ws http hillary said president news cbs new says watch republican just gop campaign obama election
Topic #4:
percent cruz ted quickly twitter compared pope internet election wife users support francis black voting actor ceo economic 42 bowl
Topic #5:
sanders bernie demdebate mitt romney caucuses malley heated president conway kellyanne cause iowa debate used called michigan asked wage strong
Topic #6:
ready rights human play tear tactics apart false chief comeback columbia civil communities way harlem loser fully pass

## Ensemble & Bagging (Bootstrap AGgregating)

In [104]:
# Nope... http://machinelearningmastery.com/ensemble-machine-learning-algorithms-python-scikit-learn/

## So text extraction + ..?

In [108]:
# http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

In [48]:
# Task 1: Load the texts
import pandas as pd
import glob, os         # for reading all .txt files
import csv
import numpy as np
from textblob import TextBlob # use kernel Python[Root]

cutoff = 436

# Read sample texts
sample = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/sample_data_results.csv') 

# Lemmatize
new_sample = pd.DataFrame(columns=("Input.content", "Answer.sentiment"))
i=0
for text in sample['Input.content']:
    blob = TextBlob(text)
    newtexts = ""

    for sentence in blob.sentences:
        newtext = ""
#        print sentence.dict

        for word in sentence.words:
            newtext += " " + word.lemmatize('v') # 'v' for 'verb'

        newtexts += newtext
        new_sample['texts'].loc[i] = newtexts
        i += 1

i=0
for answer in sample['Answer.sentiment']: # updating answers
    new_sample['Answer.sentiment'].loc[i] = answer
    i += 1
    
sample = new_sample

In [49]:
train = pd.DataFrame({'label':sample['Answer.sentiment'][:cutoff], 'texts':sample['Input.content'][:cutoff]})
test  = pd.DataFrame({'label':sample['Answer.sentiment'][cutoff:], 'texts':sample['Input.content'][cutoff:]})

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Task 4: feature engineering
for h in range(1,30): # min_df
    for i in range(h+5,100): # max_df
        for j in range(0,2): # stop_words: k=0 'engl' k=1 none
            for k in range(0,2): # k=0 CountVectorizer (count), k=1 TfidfVectorizer (weighed)
                for l in range(0,3): # l=0 MultinomialNB, l=1 GaussianNB, l=2 BernoulliNB              

                    if j==0 & k==0:
                        tf_vectorizer = CountVectorizer(max_df=i/100.0, min_df=h/100.0, stop_words='english', analyzer = "word")
                    elif j==1 & k==0:
                        tf_vectorizer = CountVectorizer(max_df=i/100.0, min_df=h/100.0, analyzer = "word")
                    elif j==0 & k==1:
                        tf_vectorizer = TfidfVectorizer(max_df=i/100.0, min_df=h/100.0, stop_words='english', analyzer = "word")
                    elif j==1 & k==1:
                        tf_vectorizer = TfidfVectorizer(max_df=i/100.0, min_df=h/100.0, analyzer = "word")

                    train_tf_ = tf_vectorizer.fit_transform(train['texts'].values)
                    test_tf_  = tf_vectorizer.transform(test['texts'].values)

                    if l==0:
                        clf = MultinomialNB()
                    elif l==1:
                        clf = GaussianNB()
                    elif l==2:
                        clf = BernoulliNB()

                    if l==0 | l==2:
                        clf.fit(train_tf_, train['label'])
                        print "%d %d %d %d %d - %.4f" % (h,i,j,k,l,clf.score(test_tf_, test['label']))
                    elif l==1:
                        clf.fit(train_tf_.toarray(), train['label'])
                        print "%d %d %d %d %d - %.4f" % (h,i,j,k,l,clf.score(test_tf_.toarray(), test['label']))

1 6 0 0 1 - 0.7778
1 6 0 0 2 - 0.6389
1 6 0 1 1 - 0.7778
1 6 0 1 2 - 0.6389
1 6 1 0 1 - 0.7778
1 6 1 0 2 - 0.6389
1 6 1 1 1 - 0.7222
1 6 1 1 2 - 0.6944
1 7 0 0 1 - 0.8056
1 7 0 0 2 - 0.6389
1 7 0 1 1 - 0.8056
1 7 0 1 2 - 0.6389
1 7 1 0 1 - 0.8056
1 7 1 0 2 - 0.6389
1 7 1 1 1 - 0.7222
1 7 1 1 2 - 0.6389
1 8 0 0 1 - 0.8056
1 8 0 0 2 - 0.6389
1 8 0 1 1 - 0.8056
1 8 0 1 2 - 0.6389
1 8 1 0 1 - 0.8056
1 8 1 0 2 - 0.6389
1 8 1 1 1 - 0.7222
1 8 1 1 2 - 0.6389
1 9 0 0 1 - 0.8056
1 9 0 0 2 - 0.5833
1 9 0 1 1 - 0.8056
1 9 0 1 2 - 0.5833
1 9 1 0 1 - 0.8056
1 9 1 0 2 - 0.5833
1 9 1 1 1 - 0.7222
1 9 1 1 2 - 0.6389
1 10 0 0 1 - 0.8056
1 10 0 0 2 - 0.6389
1 10 0 1 1 - 0.8056
1 10 0 1 2 - 0.6389
1 10 1 0 1 - 0.8056
1 10 1 0 2 - 0.6389
1 10 1 1 1 - 0.7222
1 10 1 1 2 - 0.6944
1 11 0 0 1 - 0.8056
1 11 0 0 2 - 0.6389
1 11 0 1 1 - 0.8056
1 11 0 1 2 - 0.6389
1 11 1 0 1 - 0.8056
1 11 1 0 2 - 0.6389
1 11 1 1 1 - 0.7222
1 11 1 1 2 - 0.6944
1 12 0 0 1 - 0.8056
1 12 0 0 2 - 0.6389
1 12 0 1 1 - 0.8056
1 12 0 1 2 -

KeyboardInterrupt: 